## 2020 4年　上肢動作推定における学習サンプル数と識別精度の関係


In [2]:
import time
import MyFunction

### 1. getIEMGListメソッド

11動作のデータを全波整流か半波整流をして平滑化したIEMGデータをlist型で取得する。
返り値::iemgList[gesture][trial][sensor][sample]

gesture：11動作の番号。0~10で指定する。

trial：
計測回数の番号。各動作20回計測したから0~19で指定するが、いづれかの動作の0から数えて19番目のデータが計測できていないから除外したほうがいい。
   
sensor：計測に使用したMyoアームウェアには筋電位センサが８つ付いており、各センサー番号を指定する。0~7で指定する。

sample：サンプリングレート0.001sで2秒間計測したため、cut=Falseの場合はインデックスの数は0-199。
cut=Trueで feature=Trueの場合はインデックスの数は0-153。
cut=Trueで feature=Falseの場合はインデックスの数は0-150。

In [6]:
featureF = MyFunction.getIEMGList(rectification='full', feature=True)
print('getIEMGList finish')

getIEMGList finish


### 2. featureExtractionListメソッド

getIEMGListで取得した特徴抽出用データから、MAVとMAV1の特徴抽出手法で特徴量を抽出する。
返り値::
mavIEMG[gesture][trial][sensor][sample]：IEMGをMAVで特徴抽出した特徴量
mav1IEMG[gesture][trial][sensor][sample]：IEMGをMAV1で特徴抽出した特徴量
gesture：11動作の番号。0-10で指定する。

trial：計測回数の番号。各動作20回計測したが20回目のデータが計測ミスを起こしているから除外して、0-18でインデックスを指定。

sensor：計測に使用したMyoアームウェアには筋電位センサが８つ付いており、各センサー番号を指定する。0-7で指定する。

sample：インデックスを0-149で指定する。

In [7]:
mavIEMGF, mav1IEMGF = MyFunction.featureExtractionList(featureF)
print('featureExtractionList finish')

featureExtractionList finish


### 3.学習と評価

In [4]:
modelScore = [[], [], [], [], []]
start_time = time.time()
for number in [1, 2, 3, 4, 5]:
    print('learn number : {}'.format(number))
    # 全波整流のmavを学習データとテストデータに分割
    learnMAVF, testMAVF, gesLearn, gesTest = MyFunction.makeLearnTestData(mavIEMGF, number=number)
    mavFullGestureScore, mavFullModelScore, mavGestureRate = MyFunction.gestureScore(learnMAVF, testMAVF, gesLearn, gesTest)
    
    for model in range(5):
        modelScore[model].append(mavFullModelScore[model])
    print('計測開始から {} s\n'.format(time.time()-start_time))

learn number : 1
[13]
[0, 3, 5, 6, 7, 8, 9, 12, 14, 18]
linear
交差検証 36/36  1.0

Best score on validation set: 1.0
Best parameters: {'gamma': 0.001, 'C': 0.001}
Best gesture score: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


poly
交差検証 36/36  1.075757575757575

Best score on validation set: 1.0
Best parameters: {'gamma': 0.001, 'C': 10}
Best gesture score: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


rbf
交差検証 36/36  0.22000000000000003

Best score on validation set: 1.0
Best parameters: {'gamma': 0.001, 'C': 0.1}
Best gesture score: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


svm_linear



,ges1,ges2,ges3,ges4,ges5,ges6,ges7,ges8,ges9,ges10,ges11
ges1,0.906,0.041,0.053,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
ges2,0.215,0.471,0.283,0.000,0.000,0.001,0.0,0.000,0.0,0.023,0.007
ges3,0.000,0.000,1.000,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
ges4,0.000,0.000,0.000,1.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
ges5,0.000,0.000,0.000,0.362,0.635,0.000,0.0,0.003,0.0,0.000,0.000
ges6,0.000,0.002,0.023,0.000,0.000,0.972,0.0,0.000,0.0,0.003,0.000
ges7,0.000,0.000,0.000,0.000,0.000,0.000,1.0,0.000,0.0,0.000,0.000
ges8,0.000,0.000,0.000,0.273,0.000,0.000,0.0,0.727,0.0,0.000,0.000
ges9,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,1.0,0.000,0.000
ges10,0.000,0.000,0.403,0.000,0.000,0.000,0.0,0.000,0.0,0.597,0.000


テストデータの識別精度: 0.8463030303030303
動作スコア: [0.906, 0.4713333333333333, 1.0, 1.0, 0.6353333333333333, 0.972, 1.0, 0.7273333333333334, 1.0, 0.5973333333333334, 1.0]

svm_poly



,ges1,ges2,ges3,ges4,ges5,ges6,ges7,ges8,ges9,ges10,ges11
ges1,0.794,0.047,0.104,0.000,0.000,0.000,0.034,0.000,0.0,0.000,0.021
ges2,0.253,0.433,0.268,0.000,0.000,0.000,0.000,0.000,0.0,0.045,0.000
ges3,0.000,0.000,0.994,0.000,0.000,0.000,0.000,0.000,0.0,0.006,0.000
ges4,0.011,0.000,0.000,0.753,0.000,0.000,0.000,0.000,0.0,0.000,0.236
ges5,0.005,0.000,0.000,0.338,0.641,0.000,0.000,0.011,0.0,0.000,0.005
ges6,0.000,0.000,0.000,0.000,0.000,0.951,0.000,0.000,0.0,0.049,0.000
ges7,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.0,0.000,0.000
ges8,0.000,0.000,0.000,0.384,0.000,0.000,0.000,0.616,0.0,0.000,0.000
ges9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,0.000,0.000
ges10,0.000,0.000,0.303,0.000,0.000,0.000,0.000,0.000,0.0,0.697,0.000


テストデータの識別精度: 0.8071515151515152
動作スコア: [0.794, 0.43333333333333335, 0.994, 0.7526666666666667, 0.6413333333333333, 0.9506666666666667, 1.0, 0.616, 1.0, 0.6966666666666667, 1.0]

svm_rbf



...

テストデータの識別精度: 0.9430909090909091
動作スコア: [0.9313333333333333, 0.7713333333333333, 0.9673333333333334, 0.982, 1.0, 0.9606666666666667, 1.0, 0.9906666666666667, 1.0, 0.7706666666666667, 1.0]

LDA
Score on validation set: 0.9567272727272726
Gesture score: [0.96936628 0.83402107 0.97458393 1.         0.91471886 0.99331776
 1.         0.9465174  1.         0.89206711 1.        ]


,ges1,ges2,ges3,ges4,ges5,ges6,ges7,ges8,ges9,ges10,ges11
ges1,0.913,0.026,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.061,0.000
ges2,0.072,0.573,0.248,0.000,0.000,0.000,0.000,0.000,0.0,0.033,0.074
ges3,0.000,0.000,0.981,0.000,0.000,0.000,0.005,0.000,0.0,0.014,0.000
ges4,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
ges5,0.000,0.000,0.000,0.057,0.943,0.000,0.000,0.000,0.0,0.000,0.000
ges6,0.000,0.021,0.023,0.000,0.000,0.949,0.000,0.000,0.0,0.000,0.007
ges7,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.0,0.000,0.000
ges8,0.000,0.000,0.000,0.073,0.000,0.000,0.000,0.913,0.0,0.014,0.000
ges9,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,0.000,0.000
ges10,0.000,0.000,0.225,0.000,0.000,0.000,0.000,0.000,0.0,0.743,0.033


テストデータの識別精度: 0.9103636363636364
動作スコア: [0.9133333333333333, 0.5726666666666667, 0.9806666666666667, 1.0, 0.9426666666666667, 0.9486666666666667, 1.0, 0.9133333333333333, 1.0, 0.7426666666666667, 1.0]

knn
Score on validation set: 1.0
Gesture score: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


,ges1,ges2,ges3,ges4,ges5,ges6,ges7,ges8,ges9,ges10,ges11
ges1,0.937,0.054,0.009,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
ges2,0.015,0.763,0.158,0.000,0.000,0.020,0.0,0.000,0.0,0.041,0.003
ges3,0.000,0.001,0.989,0.000,0.000,0.000,0.0,0.000,0.0,0.010,0.000
ges4,0.000,0.000,0.000,0.836,0.164,0.000,0.0,0.000,0.0,0.000,0.000
ges5,0.000,0.000,0.000,0.001,0.999,0.000,0.0,0.000,0.0,0.000,0.000
ges6,0.000,0.047,0.027,0.000,0.000,0.925,0.0,0.000,0.0,0.000,0.000
ges7,0.000,0.000,0.000,0.000,0.000,0.000,1.0,0.000,0.0,0.000,0.000
ges8,0.000,0.000,0.000,0.024,0.000,0.000,0.0,0.976,0.0,0.000,0.000
ges9,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,1.0,0.000,0.000
ges10,0.000,0.062,0.119,0.000,0.000,0.000,0.0,0.000,0.0,0.813,0.006


テストデータの識別精度: 0.9307878787878788
動作スコア: [0.9366666666666666, 0.7626666666666667, 0.9893333333333333, 0.836, 0.9993333333333333, 0.9253333333333333, 1.0, 0.976, 1.0, 0.8133333333333334, 1.0]

計測開始から 1216.8902456760406 s

